### Import Library

In [7]:
import numpy as np
import cv2
import os, sys
import time
import operator
from string import ascii_uppercase
import tkinter as tk
from PIL import Image, ImageTk
from hunspell import Hunspell
import enchant
from keras.models import model_from_json

ModuleNotFoundError: No module named 'hunspell'

### Load Model

In [2]:
# Load the main model
model_path = "../models/final_model.keras"
loaded_model = tf.keras.models.load_model(model_path)
print("Model loaded successfully")

# Load the spelling checker model
spelling_checker_model_path = "../models/model_spellingchacker/modelSpellChacker.h5"
spelling_checker_model = tf.keras.models.load_model(spelling_checker_model_path)
print("Spelling checker model loaded successfully")

# Load spelling data (if necessary)
spelling_data_path = "../data/spelling/autocorrect_dataset.csv"
spelling_data = pd.read_csv(spelling_data_path)

Model loaded successfully
Spelling checker model loaded successfully


In [3]:
# Daftar huruf sesuai label model
label_map = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J",
             "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T",
             "U", "V", "W", "X", "Y", "Z"]

# Coba buka kamera dengan ID 0 (kamera default)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Kamera tidak terdeteksi!")
else:
    print("Kamera berhasil terhubung!")

Kamera berhasil terhubung!


In [5]:
# Initialize Tkinter
root = tk.Tk()
panel = tk.Label(root)
panel.pack()

panel2 = tk.Label(root, text="", font=("Courier", 50))
panel2.pack()
panel3 = tk.Label(root, text="", font=("Courier", 40))
panel3.pack()
T = tk.Label(root, text="", font=("Courier", 40))
T.pack()

cap = cv2.VideoCapture(0)

current_symbol = " "
word = ""
blank_flag = 0
str_text = ""
ct = {chr(i): 0 for i in range(65, 91)}

def preprocess_frame(frame):
    if len(frame.shape) == 3:
        img = cv2.resize(frame, (180, 180))
    else:
        img = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
        img = cv2.resize(img, (180, 180))
    img_array = np.array(img, dtype=np.float32)
    img_array = img_array / 255.0
    img_array = np.reshape(img_array, (1, 180, 180, 3))
    return img_array

def predict(img):
    global current_symbol, word, blank_flag, str_text, ct

    try:
        result = loaded_model.predict(img)
        if result.shape[1] != 26:
            raise ValueError("Unexpected model output shape")

        prediction = {
            chr(65 + i): result[0][i] for i in range(26)
        }

        # Sort predictions in descending order
        prediction = sorted(prediction.items(), key=lambda x: x[1], reverse=True)
        current_symbol = prediction[0][0]
        confidence = prediction[0][1] * 100

        # Update letter count and word
        if current_symbol == " ":
            for i in ct:
                ct[i] = 0
        ct[current_symbol] += 1

        if ct[current_symbol] > 60:
            if current_symbol != " ":
                word += current_symbol
            ct = {chr(i): 0 for i in range(65, 91)}

        if current_symbol == " ":
            if blank_flag == 0:
                blank_flag = 1
                if len(str_text) > 0 and str_text[-1] != " ":
                    str_text += " "
                str_text += word
                word = ""
        else:
            blank_flag = 0

        # Update GUI with the predictions
        panel2.config(text=current_symbol, font=("Courier", 50))
        panel3.config(text=word, font=("Courier", 40))
        T.config(text=str_text, font=("Courier", 40))

        return confidence

    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return 0

def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.flip(frame, 1)

        # Define the central bounding box
        h, w, _ = frame.shape
        box_size = 300
        center_x, center_y = w // 2, h // 2
        x1, y1 = center_x - box_size // 2, center_y - box_size // 2
        x2, y2 = center_x + box_size // 2, center_y + box_size // 2

        # Draw the bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Extract the region of interest (ROI) for hand detection
        roi = frame[y1:y2, x1:x2]

        # Preprocess the ROI for prediction
        img_array = preprocess_frame(roi)
        confidence = predict(img_array)  # Get prediction confidence

        # Draw rectangle around detected face and put the confidence percentage
        cv2.putText(frame, f"{current_symbol}: {confidence:.2f}%", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img_tk = ImageTk.PhotoImage(image=img)

        # Update the Tkinter panel with the new image
        if panel.winfo_exists():
            panel.config(image=img_tk)
            panel.image = img_tk

    root.after(10, update_frame)

def correct_spelling(word):
    corrected_word = str(TextBlob(word).correct())
    return corrected_word

# Correct the final word and text
word = correct_spelling(word)
str_text = " ".join([correct_spelling(w) for w in str_text.split()])

# Update GUI with corrected text
panel3.config(text=word, font=("Courier", 40))
T.config(text=str_text, font=("Courier", 40))

update_frame()
root.mainloop()

# Update the frame every few milliseconds
def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.flip(frame, 1)

        h, w, _ = frame.shape
        box_size = 300
        center_x, center_y = w // 2, h // 2
        x1, y1 = center_x - box_size // 2, center_y - box_size // 2
        x2, y2 = center_x + box_size // 2, center_y + box_size // 2

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Extract ROI for hand detection
        roi = frame[y1:y2, x1:x2]

        # Preprocess ROI for prediction
        img_array = preprocess_frame(roi)

        # Use threading for prediction
        threading.Thread(target=predict, args=(img_array,)).start()

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img_tk = ImageTk.PhotoImage(image=img)

        # Update Tkinter panel
        if panel.winfo_exists():
            panel.config(image=img_tk)
            panel.image = img_tk

    root.after(1, update_frame)

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 